<div class="alert alert-block alert-success">
    <h1>DataLab HvA - serie Machine Learning Python - Tech report 12</h1>
</div>

<div class="alert alert-block alert-warning">
    <h2>Support Vector Machines - Classificatie</h2>
</div>

### 0. Doel van deze les
Vanwege de serie lessen waarbinnen deze les valt is het hierin de insteek is om het gemaakte model te gaan vergelijken met andere andere Machine Learning modellen en daarbij een kwaliteitscriterium te hanteren waarmee *alle* modellen onderling vergelijkbaar zijn. Dit is de reden dat we werken met een test- en een trainset van onze data.
**Je leert** 
- hoe je in Python met behulp van het package `sklearn` een Support Vector Machine (SVM) t.b.v. een Classificatietaak kunt maken (ook wel Support Vector Classifier - SVC)
- welke hyperparameters er gekozen kunnen worden om het model te optimaliseren
- hoe je het gevonden model kunt weergeven
- hoe je dit model kunt gebruiken om voorspellingen te genereren op nieuwe data

**Je leert niet:**
- wat de preciese wiskundige definitie en werking van het algoritme is
- hoe je met het package `seaborn` eerste visuele verkenningen van de (variatie in de) data kan uitvoeren, zie hiervoor *Tech report 1: Introductie Machine Learning* uit deze serie.
- hoe je data importeert en inlaadt in de python sessie, zie hiervoor echter wel [deze instructie](https://www.youtube.com/watch?v=1oBVx7pyuXo&list=PLwkTCAI_gJjW10xaAJpdqeQx7EQgFolfG&index=10) 
- wat het preciese concept en DE werking van K-fold crossvalidatie is, zie hiervoor *Tech report 6: K-fold crossvalidatie* uit deze serie.

**Machine Learning framework**
- Om een consistente leerlijn in de serie Machine Learning aan te houden kiezen we telkens voor een opzet waarin een inititeel model bepaald wordt o.b.v. een *deel* van de beschikbare data: de *train data*. Om het model te kunnen vergelijken met ándere modeltypen (waarbij andere algoritmen worden gebruikt) stellen we de voorspellende prestaties vast a.h.v. de nog niet gebruikte data: de *test data*. 

### 1. Laden van de benodigde packages
Packages die je zelf nog niet hebt geinstalleerd dien je vooraf in Python te installeren. 

In [1]:
# Voor de dataset en plots
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns

# Voor het bewerken en inspecteren van de data
import pandas as pd

# Voor het afsplitsen van een train- en test dataset
from sklearn.model_selection import train_test_split

# Voor het modelleren
from sklearn import svm

### 2. Laden en prepareren van de data

We gebruiken als voorbeeld een dataset `'breast_cancer'`. Het betreft data van kenmerken afkomstig van observaties van weefselonderzoek van een groep vrouwen. De kenmerken (*features*, de data waarmee we willen voorspellen) hebben betrekking op omvang van bepaalde typen en celwaarden van dit onderzochte weefsel. De categoriën die we willen voorspellen (de *targets*) zijn de 2 uitkomsten die mogelijk zijn: goedaardig (`'benign'`) of kwaadaardig (`'malignant'`). Zie [hier](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data) voor meer informatie over deze dataset.

#### Laden en weergave data
Hieronder laden we de data en geven enkele kenmerken van de set weer. 

- NB: in dit voorbeeld is géén sprake van een categoriale variabele in de verklarende data (*features*), waardoor we *géén* **one-hot-encoding** hoeven toe te passen. Dit zou i.h.a. *wél* het geval kunnen zijn. In *Tech Report 7 - Regressiebomen* een voorbeeld waar dit wordt voorgedaan, in de SVM's voor classificatie zou dit precies analoog moeten worden uitgevoerd.  
- De data is hieronder ook al meteen verdeeld over de verklarende variabelen (`X`) en de te voorspellen variabelen (`y`).

In [2]:
# Laden van een standaard dataset uit package scikit learn
cancer = datasets.load_breast_cancer()

X = pd.DataFrame(cancer.data, columns = cancer.feature_names)
y = pd.Series(cancer.target)
print(X.head())
print(X.shape)
print(X.dtypes)
print("de targets, weergegeven als [0, 1], komen respectievelijk overeen met", cancer.target_names)
print(y.shape)
print(y.dtypes)

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \
0           

#### Afsplitsen test data en train data
Voor meer details over deze stap, zie *Tech report 2: Datapreparatie* uit deze serie.

In [3]:
# Split in training en test data 
# random_state kiezen zorgt voor reproduceerbaarheid van dit resultaat - gebruik dat ! Een andere waarde is uiteraard ook prima
# we kiezen voor een verhouden train - test van 70% - 30%. Andere keuzen kunnen ook!
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.30,random_state=999)
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(398, 30)
(171, 30)
(398,)
(171,)


### 3. Voorbeschouwing
Zomaar modelleren doe je niet, eerst moeten we vaststellen wat we willen bereiken of welke vraag we zouden willen beantwoorden. Voor deze les zal dat worden:
- Taak: stel een SVC op om de klasse waartoe de het onderzochte weefsel behoort zo goed mogelijk te voorspellen op basis van de overige variabelen 

Vanuit de taak zullen we dus stellen:
- De *onafhankelijke* variabelen ($x_i$) zijn de 30 variabelen `'mean radius'`,` 'mean texture'`, ... ,`'worst symmetry'`,` 'worst fractal dimension'`
- De *afhankelijke* variabele ($y$) is de klasse die het type weefsel reprenteert. De target variabele wordt weergegeven als `0` of `1`, welke respectievelijk overeenkomen met klassenamen `'malignant'` en `'benign'`.

#### Support Vector Machine - Classificatie
Het algoritme is gebaseerd op het zoeken van zogenaamde *hyperplanes* in de *feature space* die de data lineair scheidt tussen de te onderscheiden target labels in de betreffende classificatietaak. Een sterk punt is dat het juist heel goed overweg kan met data die in eerste instantie juist *niet* goed lineair te scheiden is door de data *eerst* te transformeren naar een *hogere dimensie* waarin de lineaire scheiding *wel* (of: veel beter) mogelijk is. Die transformatie wordt de *kernel trick* genoemd. Hierbij verwijst de *kernel* naar een hyperparameter die dient te worden gekozen en de wijze waarop de data wordt getransformeerd bepaald. De uiteindelijk gevonden optimale *hyperplanes* worden vastgelegd door een beperkt aantal *support vectors* (een door het algoritme gevonden optimale set waarnemingen uit de data, liefst zo klein mogelijk, die ieder toebehoren aan één van de target labels), hieraan ontleent dit algoritme zijn naam. Het SVM algoritme wordt wat uitvoeriger toegelicht op [deze](https://scikit-learn.org/stable/modules/svm.html#) pagina, maar ook [hier](https://www.datacamp.com/tutorial/svm-classification-scikit-learn-python) vind je een (misschien iets toegankelijkere?) uitleg. 

### 4. Model schatten
#### lees documentatie bij package
Voordat we een model gaan schatten is het van belang dat je de documentatie van het algoritme dat we uit het package gaan gebruiken lezen. In dit geval is dat dus voor `svm.SVC()`, die lees je bijvoorbeeld online [hier](https://scikit-learn.org/stable/modules/svm.html#classification). Alternatief is om de `help()` functie te gebruiken, zie hieronder. 

In [4]:
# We gaan de methode svm.SVC() uit het sklearn package gebruiken, inspecteer hiervoor de documentatie. Dat kan 
help(svm.SVC())

Help on SVC in module sklearn.svm._classes object:

class SVC(sklearn.svm._base.BaseSVC)
 |  SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)
 |  
 |  C-Support Vector Classification.
 |  
 |  The implementation is based on libsvm. The fit time scales at least
 |  quadratically with the number of samples and may be impractical
 |  beyond tens of thousands of samples. For large datasets
 |  consider using :class:`~sklearn.svm.LinearSVC` or
 |  :class:`~sklearn.linear_model.SGDClassifier` instead, possibly after a
 |  :class:`~sklearn.kernel_approximation.Nystroem` transformer or
 |  other :ref:`kernel_approximation`.
 |  
 |  The multiclass support is handled according to a one-vs-one scheme.
 |  
 |  For details on the precise mathematical formulation of the provided
 |  kernel functions and how `

Zie bijvoorbeeld de parameters: `C`, `kernel`, `degree`, en `gamma`. Ook lees je dat er een *method*  `fit()` voor deze *object class* gedefinieerd is om het model te *fitten*.

#### model fitten
We gebruiken dus de train data om een model te fitten. Dit is, zoals je ziet, een eenvoudig regeltje code. We geven het model een naam (`svc`) zodat we het later weer opnieuw kunnen aanroepen / gebruiken.

In [5]:
# Met model fitten default van package
svc = svm.SVC(random_state=0)
#Fit the regressor object to the dataset.
svc.fit(train_X,train_y)

SVC(random_state=0)

#### Model weergeven
De mogelijkheden voor het weergeven van (kenmerken van) een gevonden SVM zijn wat beperkt. Bijvoorbeeld het *aantal* benodigde support vectors zegt iets. Hoe meer, hoe complexer het model, hoe groter de kans op overfitting op de trainingsdata.

In [6]:
n_supp_vecs = svc.n_support_
print("Aantal support vectoren per target label [0 1]: ", n_supp_vecs )

Aantal support vectoren per target label [0 1]:  [52 51]


#### Modelscore op train-/test dataset
We zien hieronder dat er sprake is van een (enigszins) overfit model. 

In [7]:
predicted_y_tr = svc.predict(train_X)
predicted_y = svc.predict(test_X)
from sklearn import metrics

# accuracy op trainset
print("De accuracy op de trainset = ", metrics.accuracy_score(train_y, predicted_y_tr))

# accuracy op testset
print("De accuracy op de testset = ", metrics.accuracy_score(test_y, predicted_y))

De accuracy op de trainset =  0.9221105527638191
De accuracy op de testset =  0.8771929824561403


### 5. Standaardiseren X-data
Voor een SVM is het aan te raden de numerieke variabelen van de onafhankelijke data te standaardiseren. Lees hierover meer in *Tech Report 2 - Datapreparatie* (sectie 4) uit deze serie. In de stappen hieronder doorlopen we dit proces en zal duidelijk worden dat de prestaties verbeterd zijn.

In [8]:
#Inladen method voor standaardiseren
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_X) #Let op: de scaler wordt gefit op de traindataset

#herschalen van de X-data uit de trainset
train_X_scaled = pd.DataFrame(scaler.transform(train_X), columns = cancer.feature_names)
print(train_X_scaled.head())

#herschalen van de X-data uit de testset - NB: op basis van de scaler die is gefit op de traindata!
test_X_scaled = pd.DataFrame(scaler.transform(test_X), columns = cancer.feature_names)


   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0    -0.847325     -0.585584       -0.879856  -0.774499        -1.017374   
1    -0.062390     -0.688816       -0.131992  -0.164532        -0.605482   
2     1.094959      0.522271        1.354622   0.983962         1.869354   
3    -0.122549      0.046397       -0.093045  -0.217323        -0.387011   
4    -0.188438      0.172289       -0.243032  -0.245727        -0.393273   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0         -1.245801       -0.922753            -0.891124      -1.050320   
1         -0.926851       -0.906295            -0.705219      -1.366239   
2          2.972118        2.853962             1.856031       1.135700   
3          0.223141        0.102817             0.000083       0.173416   
4         -0.773723       -0.367277            -0.355675      -0.734400   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \
0           

#### Model nogmaals fitten - nu op geschaalde data

In [9]:
# Met model fitten default van package
svc2 = svm.SVC(random_state=0)
#Fit the regressor object to the dataset.
svc2.fit(train_X_scaled,train_y)

SVC(random_state=0)

#### Model weergeven
We zien dat we een model met *minder* support vectors krijgen, dat duidt erop dat de feature space met een *minder complex* gescheiden kon worden!

In [10]:
n_supp_vecs = svc2.n_support_
print("Aantal support vectoren per target label [0 1]: ", n_supp_vecs )

Aantal support vectoren per target label [0 1]:  [47 49]


#### Modelscore svc2 op train-/test dataset
We zien dat de prestaties van dit model aanmerkelijk zijn verbeterd door de standaardisatie van de X-data en dat ook de mate van overfitting sterk is verminderd!

In [11]:
predicted_y_tr2 = svc2.predict(train_X_scaled)
predicted_y2 = svc2.predict(test_X_scaled)
from sklearn import metrics

# accuracy op trainset
print("De accuracy op de trainset = ", metrics.accuracy_score(train_y, predicted_y_tr2))

# accuracy op testset
print("De accuracy op de testset = ", metrics.accuracy_score(test_y, predicted_y2))

De accuracy op de trainset =  0.9824120603015075
De accuracy op de testset =  0.9707602339181286


### 6. Tuning hyperparameters
We hebben onze boom uit het voorbeeld nu gefit voor een model dat de default hyperparameters meegekregen heeft. Verderop onderzoeken we uitkomsten bij andere keuzen van de hyperparameters. 

Het model (ook het tweede) is zo te zien wat overfit op de trainingsdata. Dit hoeft niet per se een probleem te zijn, maar het is wel van belang om te onderzoeken of het model met ándere hyperparameters wellicht betere scoreresultaten behaalt. Gebruikelijk is dit te doen in een K-fold Cross Validatie, zie hiervoor *Tech report 6: K-fold crossvalidatie* uit deze serie. 

Parameters om sowieso te onderzoeken zijn `C`, `kernel` (kies bijvoorbeeld `'poly'` en `'rbf'`) en dan  daarbij `degree` (bij `'poly'`) respectievelijk `gamma` (bij `'rbf'`) een waardebereik telaten doorlopen. 

#### hyperparameters
Nu gaan we ons bij wijze van voorbeeld richten op een aantal *hyperparameters* die we willen kiezen om wellicht tot beter modelprestaties te komen:  
1. De Regularization parameter van ons model, deze bepaalt de mate van misclassificatie die we willen vermijden: `C = .. .`  
2. De kernel, deze bepaalt de wiskundige transformatie van de data: `kernel = ...`  
3. Bij elke kernel hoort weer een eigen parameter, we doorlopen 
    - Voor de `kernel = 'poly'` de parameter `degree`
    - Voor de `kernel = 'rbf'` de parameter `gamma`

#### Grid search
Hieronder zien we hoe we een *grid*  definieren van modelparameters die we doorlopen en bij elke combinatie wordt een score bepaald o.b.v. een Cross-Validatie (k = 5). We doorlopen de grids voor de beide keuzes van de *kernel* apart. Misschien is dit niet strikt nodig, maar het lijkt wat logischer vanwege de parameters *degree* resp. *gamma* die eraan gekoppled zijn.

We gebruiken de geschaalde traindata.

In [12]:
# Hyper parameters range om te doorlopen
# Dit is slechts een voorbeeld, ook de ándere modelparameters kunn hierin worden betrokken
import numpy as np

parameters_poly={"kernel" : ["poly"], 
                "degree" : np.arange(1,10),
               "C" : 10. ** np.arange(-3, 8)
           }

parameters_rbf={"kernel" : ["rbf"],               # NB: dit is de default
            "gamma" : 10. ** np.arange(-5, 4),
               "C" : 10. ** np.arange(-3, 8)
           }

In [13]:
# Importeer vanuit scikit learn package de tooling:
from sklearn.model_selection import GridSearchCV

We starten hieronder de grid search. We maken tweemaal een `tuning_model` waarin alle resultaten van de getrainde modellen op alle gridpunten zullen worden bewaard. Zie het *Tech report 6 K-fold crossvalidatie en Grid Search* in deze serie voor meer uitleg en voorbeelden.

Eerst volgt een definitie van de grid search. We zetten (om de output in dit notebook niet te laten exploderen) `verbose = 1`, maar dit kun je zélf eventueel aanpassen.

In [14]:
svc_grid_poly = svm.SVC(random_state=0, coef0=1) #coeff op 1 is beste keu
tuning_model_poly = GridSearchCV(estimator = svc_grid_poly, param_grid = parameters_poly,scoring='accuracy',cv=5,verbose=1)

In [15]:
svc_grid_rbf = svm.SVC(random_state=0)
tuning_model_rbf = GridSearchCV(estimator = svc_grid_rbf, param_grid = parameters_rbf,scoring='accuracy',cv=5,verbose=1)

Hieronder starten we de daarwerkelijke evaluatie, deze stap kan in sommigen gevallen best lang duren!

In [16]:
tuning_model_poly.fit(train_X_scaled,train_y)

Fitting 5 folds for each of 99 candidates, totalling 495 fits


GridSearchCV(cv=5, estimator=SVC(coef0=1, random_state=0),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04,
       1.e+05, 1.e+06, 1.e+07]),
                         'degree': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'kernel': ['poly']},
             scoring='accuracy', verbose=1)

In [17]:
tuning_model_rbf.fit(train_X_scaled,train_y)

Fitting 5 folds for each of 99 candidates, totalling 495 fits


GridSearchCV(cv=5, estimator=SVC(random_state=0),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04,
       1.e+05, 1.e+06, 1.e+07]),
                         'gamma': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03]),
                         'kernel': ['rbf']},
             scoring='accuracy', verbose=1)

Een optimum voor polynomiale kernel werd gevonden voor:

In [18]:
print("Beste model: ", tuning_model_poly.best_estimator_)
print("Beste score:  accuracy = ", tuning_model_poly.best_score_)
print("Beste parameter combi: ", tuning_model_poly.best_params_)

Beste model:  SVC(coef0=1, kernel='poly', random_state=0)
Beste score:  accuracy =  0.9774367088607594
Beste parameter combi:  {'C': 1.0, 'degree': 3, 'kernel': 'poly'}


Een optimum voor radiale kernel werd gevonden voor:

In [19]:
print("Beste model: ", tuning_model_rbf.best_estimator_)
print("Beste score:  accuracy = ", tuning_model_rbf.best_score_)
print("Beste parameter combi: ", tuning_model_rbf.best_params_)

Beste model:  SVC(C=10.0, gamma=0.01, random_state=0)
Beste score:  accuracy =  0.9774367088607594
Beste parameter combi:  {'C': 10.0, 'gamma': 0.01, 'kernel': 'rbf'}


We zien dat beide modeltypen gelijke accuracy op de grid search behalen, we vonden geen parameterwaarden op de randen, maar de stapgrootte was nogal groot, er is voldoende ruimte dit verder te finetunen, dit laten we aan jullie. 

#### Meest eenvoudige model kiezen
Als de modelprestaties (nagenoeg) geen onderscheid tussen modelkeuzes laten zien, kies dan het meest eenvoudige model. Dan doen we o.b.v. het aantal support vectoren:

In [21]:
n_supp_vecs_poly = tuning_model_poly.best_estimator_.n_support_
print("Aantal support vectoren per target label [0 1] poly: ", n_supp_vecs_poly )

n_supp_vecs_rbf = tuning_model_rbf.best_estimator_.n_support_
print("Aantal support vectoren per target label [0 1] rbf: ", n_supp_vecs_rbf )

Aantal support vectoren per target label [0 1] poly:  [23 30]
Aantal support vectoren per target label [0 1] rbf:  [23 28]


We kiezen dan de radiale kernel (`kernel = "rbf"`, de defaultwaarde).

### 7. Trainen model met optimale parameterwaarden
We trainen hieronder het model nogmaals, maar dan op de gehele train dataset en mét de gevonden optimale parameterwaarden.

In [22]:
#Definitie
svc_opt = svm.SVC(C=10.0, gamma=0.01, random_state=0) #kernel default = 'rbf'
#fitten
svc_opt.fit(train_X_scaled,train_y)


SVC(C=10.0, gamma=0.01, random_state=0)

We maken voorspelling op de train data en evalueren deze:

In [23]:
predicted_y_tr = svc_opt.predict(train_X_scaled)
from sklearn import metrics

#accuracy train
print("De accuracy op de trainset = ", metrics.accuracy_score(train_y, predicted_y_tr))

De accuracy op de trainset =  0.9899497487437185


### 7A - Opnieuw fitten niet nodig
Het model `GridSearchCV()`leverde met `tuning_model_rbf.best_estimator_` al dit optimale model (op gehele traindataset). Dus bovenstaande kun je ook vervangen door:

In [24]:
predicted_y_tr2 = tuning_model_rbf.best_estimator_.predict(train_X_scaled)
from sklearn import metrics

#accuracy train
print("De accuracy op de trainset (alt)= ", metrics.accuracy_score(train_y, predicted_y_tr2))

De accuracy op de trainset (alt)=  0.9899497487437185


### 8. Voorspellingen en performance op testset
Hieronder genereren we voorspellingen van y op de testset. Tevens produceren we een modelscore. Voor meer toelichting daarover verwijzen we naar *Tech report 3: performance metrics regressie en classificatie*.

We zien dat de mate van overfitting kleiner is, maar de prestatie op test *niet* beter dan bij het *ongelimiteerde zeer complexe model*. Dat kan gebeuren en is bijvoorbeeld een gevolg van de toevallige samenstelling van de *test data*. NB: wat cf. *CRISP-DM* nu **niet** is toegestaan is dat deze observatie je doet besluiten *opnieuw* aan de model architectuur (de hyper  parameters) te tunen! 

In [25]:
predicted_y = svc_opt.predict(test_X_scaled)

#accuracy test
print("De accuracy op de testset = ", metrics.accuracy_score(test_y, predicted_y))

De accuracy op de testset =  0.9707602339181286


De prestaties op de traindata zijn ten opzichte van het default model wel iets toegenomen, maar op test ongewijzigd gebleven t.o.v. het default model.